In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import hvplot.pandas
from scipy.stats import linregress
import json

In [4]:
file_path = "../Resources/VAERSDATA1 (3).csv"
covid_df = pd.read_csv(file_path)
covid_df.sample(20)

,VAERS_ID,RECVDATE,STATE,AGE_YRS,SEX,DIED,DATEDIED,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,V_ADMINBY,VAX_MANU,HISTORY,ALLERGIES
52901,1222587,4/17/2021,NC,17,F,NaN,NaN,NaN,3/9/2021,3/9/2021,0.0,PUB,JANSSEN,NaN,None reported
13891,2496620,11/2/2022,NaN,8,M,NaN,NaN,NaN,10/9/2022,10/9/2022,0.0,UNK,PFIZER\BIONTECH,NaN,NaN
131436,2035614,1/14/2022,MN,27,F,NaN,NaN,Y,8/30/2021,12/29/2021,121.0,UNK,PFIZER\BIONTECH,PREGNANCY OF 38 WEEKS,NO KNOWN
880645,2267697,5/9/2022,CO,90,F,NaN,NaN,N,4/25/2022,5/9/2022,14.0,PVT,MODERNA,Glaucoma,Fentinal
676051,2681064,9/11/2023,VA,66,M,NaN,NaN,N,2/13/2021,9/20/2021,219.0,UNK,PFIZER\BIONTECH,Medical History/Concurrent Conditions: BPH,NaN
857300,2370141,7/15/2022,ME,83,U,NaN,NaN,U,7/12/2022,7/12/2022,0.0,UNK,MODERNA,NaN,NaN
329107,1165905,4/4/2021,SC,43,F,NaN,NaN,N,3/16/2021,3/21/2021,5.0,PHM,PFIZER\BIONTECH,Medical History/Concurrent Conditions: Hyperte...,NaN
187218,1885462,11/19/2021,NaN,32,U,NaN,NaN,NaN,11/12/2021,11/12/2021,0.0,UNK,MODERNA,NaN,NaN
347651,1810476,10/23/2021,WA,44,F,NaN,NaN,N,7/9/2021,7/9/2021,0.0,PHM,PFIZER\BIONTECH,Medical History/Concurrent Conditions: Asthma;...,NaN
533979,2542147,12/23/2022,IN,57,F,NaN,NaN,U,11/13/2022,11/13/2022,0.0,PHM,PFIZER\BIONTECH,NaN,NaN


In [5]:
covid_df.drop(['NUMDAYS', 'V_ADMINBY', 'HISTORY', 'ALLERGIES'], axis=1, inplace=True)

covid_df.head()


,VAERS_ID,RECVDATE,STATE,AGE_YRS,SEX,DIED,DATEDIED,RECOVD,VAX_DATE,ONSET_DATE,VAX_MANU
0,1021156,2/10/2021,IL,32,F,NaN,NaN,N,1/27/2021,2/4/2021,MODERNA
1,1029567,2/14/2021,NH,61,M,NaN,NaN,U,2/14/2021,2/14/2021,MODERNA
2,1061353,2/28/2021,WA,74,M,NaN,NaN,N,2/24/2021,2/24/2021,PFIZER\BIONTECH
3,1066793,3/2/2021,AZ,81,M,NaN,NaN,N,2/12/2021,2/26/2021,PFIZER\BIONTECH
4,1219181,4/16/2021,TX,15,M,NaN,NaN,Y,4/15/2021,4/15/2021,PFIZER\BIONTECH


In [6]:
#total count of vaccines administered by manufacturer
vaccine_counts = covid_df.groupby('VAX_MANU').size().reset_index(name='Total_Vaccines_Administered')

vaccine_counts



,VAX_MANU,Total_Vaccines_Administered
0,JANSSEN,53687
1,MODERNA,408657
2,NOVAVAX,300
3,PFIZER\BIONTECH,413910
4,UNKNOWN MANUFACTURER,19975


In [7]:
# Adverse Reactions
adverse_reactions_df = covid_df[(covid_df['RECOVD'] != 'Y')]

# Seperate deaths
deaths_df = covid_df[(covid_df['DIED'] == 'Y')]

# Group by manufacturer for adverse reactions
adverse_reactions_by_manufacturer = adverse_reactions_df.groupby('VAX_MANU').size().reset_index(name='Adverse_Reactions')

# Group by manufacturer for deaths
deaths_by_manufacturer = deaths_df.groupby('VAX_MANU').size().reset_index(name='Deaths')

# Merge both summaries
summary_df = adverse_reactions_by_manufacturer.merge(deaths_by_manufacturer, on='VAX_MANU', how='outer').fillna(0)

# Print
summary_df



,VAX_MANU,Adverse_Reactions,Deaths
0,JANSSEN,35521,1317.0
1,MODERNA,268139,7563.0
2,NOVAVAX,218,0.0
3,PFIZER\BIONTECH,287050,7403.0
4,UNKNOWN MANUFACTURER,13543,623.0


In [8]:
#adverse reaction %
adverse_percentage_df = vaccine_counts.merge(adverse_reactions_by_manufacturer, on='VAX_MANU', how='left').fillna(0)

adverse_percentage_df['Adverse_Reaction_Percentage'] = (adverse_percentage_df['Adverse_Reactions'] / adverse_percentage_df['Total_Vaccines_Administered']) * 100

adverse_percentage_df

,VAX_MANU,Total_Vaccines_Administered,Adverse_Reactions,Adverse_Reaction_Percentage
0,JANSSEN,53687,35521,66.163131
1,MODERNA,408657,268139,65.614684
2,NOVAVAX,300,218,72.666667
3,PFIZER\BIONTECH,413910,287050,69.350825
4,UNKNOWN MANUFACTURER,19975,13543,67.799750


In [9]:
#death % by manufacturer

death_percentage_df = vaccine_counts.merge(deaths_by_manufacturer, on="VAX_MANU", how='left').fillna(0)

death_percentage_df['Deaths'] = death_percentage_df['Deaths'].astype(int)

death_percentage_df['Death_Percentage'] = (death_percentage_df['Deaths'] / death_percentage_df['Total_Vaccines_Administered']) * 100

death_percentage_df




,VAX_MANU,Total_Vaccines_Administered,Deaths,Death_Percentage
0,JANSSEN,53687,1317,2.453108
1,MODERNA,408657,7563,1.850696
2,NOVAVAX,300,0,0.000000
3,PFIZER\BIONTECH,413910,7403,1.788553
4,UNKNOWN MANUFACTURER,19975,623,3.118899
